# Homework 2: Word Similarity

Student Name: Indah Permatasari

Student ID: 929578

## General info

<b>Due date</b>: Thursday, 18 Apr 2019 4pm

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 7% of mark for class (with 6% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib and Scikit-Learn. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>. 

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

## Overview

In this homework, you'll be quantifying the similarity between pairs of words of a dataset using different methods with the word co-occurrence in the Brown corpus and synset structure of WordNet. Firstly, you will preprocess the dataset to filter out the rare and ambiguous words. Secondly, you will calculate the similarity scores for pairs of words in the filtered dateset using Lin similarity, NPMI and LSA. Lastly, you will quantify how well these methods work by comparing to a human annotated gold-standard.

## 1. Preprocessing (2 marks)

<b>Instructions</b>: For this homework we will be comparing our methods against a popular dataset of word similarities called <a href="http://www.cs.technion.ac.il/~gabr/resources/data/wordsim353/">Similarity-353</a>. You need to first obtain this dataset, which is available on LMS. The file we will be using is called *set1.tab*. Make sure you save this in the same folder as the notebook.  Except for the header (which should be stripped out), the file is tab formated with the first two columns corresponding to two words, and the third column representing a human-annotated similarity between the two words. <b>You should ignore the subsequent columns</b>.

Here shows the first six lines of the file:

```
Word 1	Word 2	Human (mean)	1	2	3	4	5	6	7	8	9	10	11	12	13	
love	sex	6.77	9	6	8	8	7	8	8	4	7	2	6	7	8	
tiger	cat	7.35	9	7	8	7	8	9	8.5	5	6	9	7	5	7	
tiger	tiger	10.00	10	10	10	10	10	10	10	10	10	10	10	10	10	
book	paper	7.46	8	8	7	7	8	9	7	6	7	8	9	4	9	
computer	keyboard	7.62	8	7	9	9	8	8	7	7	6	8	10	3	9	
```
    
You should load this file into a Python dictionary (NOTE: in Python, tuples of strings, i.e. ("tiger","cat") can serve as the keys of a dictionary to map to their human-annotated similarity). This dataset contains many rare words: we need to filter this dataset in order for it to be better suited to the resources we will use in this assignment. So your first goal is to filter this dataset to generate a smaller test set where you will evaluate your word similarity methods.

The first filtering is based on document frequencies in the Brown corpus, in order to remove rare words. In this homework, we will be treating the paragraphs of the Brown corpus as our "documents". You can iterate over them by using the `paras` method of the corpus reader. You should remove tokens that are not alphabetic. Tokens should be lower-cased and lemmatized. Now calculate document frequencies for each word type, and use this to remove from your word similarity data any word pairs where at least one of the two words has a document frequency of less than 8 in this corpus.

For this part, you should store all the word pair and similarity mappings in your filtered test set in a dictionary called *filtered_gold_standard*. You may check the section, <i>"For your testing"</i>, below for the expected *filtered_gold_standard*.

(1 mark)

In [363]:
import nltk
from nltk.corpus import brown
from nltk.corpus import wordnet

nltk.download("brown")
nltk.download("wordnet")

# filtered_gold_standard stores the word pairs and their human-annotated similarity in your filtered test set
filtered_gold_standard = {}

###
# Your answer BEGINS HERE
###

import os
import csv
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

# function to do lemmatisation, copied from the notebook given in subject materials
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma  

# function to count the occurence of each word type in a list
def word_counter(wordlist):
    counter = {}
    for word in wordlist:
        counter[word] = counter.get(word,0) + 1
    return counter

# function to return the counter of word type in every document in a corpus
def process(corpus):
    wordfreq_doc = []
    for doc in corpus:
        wordlist = []
        for texts in doc:
            for word in texts:
                if word.isalpha():
                    wordlist.append(lemmatize(word.lower()))
        wordfreq_doc.append(word_counter(wordlist))      
    return wordfreq_doc

brown_corpus = nltk.corpus.brown.paras()
wordfreq = process(brown_corpus)

# function to return number of documents contain a specific word
def count_word(word):
    global wordfreq
    count = 0
    for docid in range(len(wordfreq)):
        wordlist = wordfreq[docid].keys()
        if word in wordlist:
            count = count + 1
    return count

# read set1.tab in the same folder as python notebook
path = os.path.join(os.path.dirname(os.path.realpath('__file__')), "set1.tab")
with open(path, 'r') as file:
    record = csv.reader(file, delimiter='\t')
    next(record)
    for row in record:
        try:
            if count_word(row[0]) >= 8 and  count_word(row[1]) >= 8:
                filtered_gold_standard[row[0], row[1]] =  float(row[2])
        except:
            continue
            
###
# Your answer ENDS HERE
###

print(len(filtered_gold_standard))
print(filtered_gold_standard)

[nltk_data] Downloading package brown to /Users/indah/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/indah/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


94
{('love', 'sex'): 6.77, ('tiger', 'cat'): 7.35, ('tiger', 'tiger'): 10.0, ('book', 'paper'): 7.46, ('plane', 'car'): 5.77, ('train', 'car'): 6.31, ('telephone', 'communication'): 7.5, ('television', 'radio'): 6.77, ('drug', 'abuse'): 6.85, ('bread', 'butter'): 6.19, ('doctor', 'nurse'): 7.0, ('professor', 'doctor'): 6.62, ('student', 'professor'): 6.81, ('smart', 'student'): 4.62, ('smart', 'stupid'): 5.81, ('company', 'stock'): 7.08, ('stock', 'market'): 8.08, ('stock', 'phone'): 1.62, ('stock', 'egg'): 1.81, ('stock', 'live'): 3.73, ('stock', 'life'): 0.92, ('book', 'library'): 7.46, ('bank', 'money'): 8.12, ('wood', 'forest'): 7.73, ('money', 'cash'): 9.08, ('king', 'queen'): 8.58, ('bishop', 'rabbi'): 6.69, ('holy', 'sex'): 1.62, ('football', 'basketball'): 6.81, ('football', 'tennis'): 6.63, ('tennis', 'racket'): 7.56, ('law', 'lawyer'): 8.38, ('movie', 'star'): 7.38, ('movie', 'critic'): 6.73, ('movie', 'theater'): 7.92, ('space', 'chemistry'): 4.88, ('alcohol', 'chemistry'): 

<b>For your testing: </b>

In [364]:
assert(len(filtered_gold_standard) > 50 and len(filtered_gold_standard) < 100)

In [365]:
assert(filtered_gold_standard[('love', 'sex')] == 6.77)

<b>Instructions</b>: Here, you apply the second filtering. The second filtering is based on words with highly ambiguous senses and involves using the NLTK interface to WordNet. Here, you should remove any words which do not have a *single primary sense*. We define single primary sense here as either a) having only one sense (i.e. only one synset), or b) where the count (as provided by the WordNet `count()` method for the lemmas associated with a synset) of the most common sense is at least 4 times larger than the next most common sense. Note that a synset can be associated with multiple lemmas. You should only consider the count of your lemma. Also, you should remove any words where the primary sense is not a noun (this information is also in the synset). Store the synset corresponding to this primary sense in a dictionary for use in the next section. Given this definition, remove the word pairs from the test set where at least one of the words does not meet the above criteria.

When you have applied the two filtering steps, you should store all the word pair and similarity mappings in your filtered test set in a dictionary called *final_gold_standard*. You may check the section, <i>"For your testing"</i>, below for the expected *final_gold_standard*.

(1 mark)

In [366]:
# final_gold_standard stores the word pairs and their human-annotated similarity in your final filtered test set
final_gold_standard = {}

###
# Your answer BEGINS HERE
###

# function to return the status whether a word match the filter or not
# if a word match the filter it returns the primary synset of that word
def check_sense(word):
    synsets = wordnet.synsets(word)
    if len(synsets) < 1:
        return False, ''   
    elif len(synsets) == 1:
        if synsets[0].pos() == 'n':
            return True,synsets[0]
        else:
            return False,''
    else:
        sense_count = {}
        for synset in synsets:
            for lemma in synset.lemmas():
                if lemma.name().lower() == word:            
                    sense_count[synset] = lemma.count()
                    break
        topsynset = sorted(sense_count.items(), key=lambda x: x[1], reverse = True)[:2]  
        if topsynset[0][1] > 0 and topsynset[0][1] >= 4*topsynset[1][1] and topsynset[0][0].pos() == 'n':
                return True,topsynset[0][0]
        else:
            return False,''   
        
# synsets stores the primary synsets of the word pairs       
synsets = {}        

for keys in filtered_gold_standard.keys():
    check_key1, synset1 = check_sense(keys[0])
    check_key2, synset2 = check_sense(keys[1])
    if check_key1 and check_key2:
        final_gold_standard[keys] = filtered_gold_standard[keys]
        synsets[keys] = [synset1, synset2]
        
###
# Your answer ENDS HERE
###

print(len(final_gold_standard))
print(final_gold_standard)        

26
{('bread', 'butter'): 6.19, ('professor', 'doctor'): 6.62, ('student', 'professor'): 6.81, ('stock', 'egg'): 1.81, ('money', 'cash'): 9.08, ('king', 'queen'): 8.58, ('bishop', 'rabbi'): 6.69, ('football', 'basketball'): 6.81, ('football', 'tennis'): 6.63, ('alcohol', 'chemistry'): 5.54, ('baby', 'mother'): 7.85, ('car', 'automobile'): 8.94, ('journey', 'voyage'): 9.29, ('coast', 'shore'): 9.1, ('brother', 'monk'): 6.27, ('journey', 'car'): 5.85, ('coast', 'hill'): 4.38, ('forest', 'graveyard'): 1.85, ('monk', 'slave'): 0.92, ('coast', 'forest'): 3.15, ('psychology', 'doctor'): 6.42, ('psychology', 'mind'): 7.69, ('psychology', 'health'): 7.23, ('psychology', 'science'): 6.71, ('planet', 'moon'): 8.08, ('planet', 'galaxy'): 8.11}


<b>For your testing:</b>

In [367]:
assert(len(final_gold_standard) > 10 and len(final_gold_standard) < 40)

In [368]:
assert(final_gold_standard[('professor', 'doctor')] == 6.62)

## 2. Word similiarity scores with Lin similarity, NPMI and LSA (3 marks)

<b>Instructions</b>: Now you will create several dictionaries with similarity scores for pairs of words in your test set derived using the techniques discussed in class. The first of these is the Lin similarity for your word pairs using the information content of the Brown corpus, which you should calculate using the primary sense for each word derived above. You can use the built-in method included in the NLTK interface, you don't have to implement your own. 

When you're done, you should store the word pair and similarity mappings in a dictionary called *lin_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *lin_similarities*. 

(1 mark)

In [369]:
from nltk.corpus import wordnet_ic
nltk.download('wordnet_ic')

# lin_similarities stores the word pair and Lin similarity mappings
lin_similarities = {}

###
# Your answer BEGINS HERE
###

brown_ic = wordnet_ic.ic('ic-brown.dat')
for keys in final_gold_standard.keys():
    synset1, synset2 = synsets[keys]
    lin_similarities[keys] = synset1.lin_similarity(synset2, brown_ic) 
    
###
# Your answer ENDS HERE
###

print(lin_similarities)

[nltk_data] Downloading package wordnet_ic to
[nltk_data]     /Users/indah/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!


{('bread', 'butter'): 0.711420490146294, ('professor', 'doctor'): 0.7036526610448273, ('student', 'professor'): 0.26208607023317687, ('stock', 'egg'): -0.0, ('money', 'cash'): 0.7888839126424345, ('king', 'queen'): 0.25872135992145145, ('bishop', 'rabbi'): 0.6655650900427844, ('football', 'basketball'): 0.7536025025710653, ('football', 'tennis'): 0.7699955045932811, ('alcohol', 'chemistry'): 0.062235427146896456, ('baby', 'mother'): 0.6315913189894092, ('car', 'automobile'): 1.0, ('journey', 'voyage'): 0.6969176573027711, ('coast', 'shore'): 0.9632173804623256, ('brother', 'monk'): 0.24862817480738675, ('journey', 'car'): -0.0, ('coast', 'hill'): 0.5991131628821826, ('forest', 'graveyard'): -0.0, ('monk', 'slave'): 0.2543108201944307, ('coast', 'forest'): -0.0, ('psychology', 'doctor'): -0.0, ('psychology', 'mind'): 0.304017384194818, ('psychology', 'health'): 0.06004979886905243, ('psychology', 'science'): 0.8474590505736942, ('planet', 'moon'): 0.7808882364067532, ('planet', 'galaxy'

<b>For your testing:</b>

In [370]:
assert(lin_similarities[('professor', 'doctor')] > 0.5 and lin_similarities[('professor', 'doctor')] < 1)

**Instructions:** Next, you will calculate Normalized PMI (NPMI) for your word pairs using word frequency derived from the Brown.

PMI is defined as:

\begin{equation*}
PMI = \log_2\left(\frac{p(x,y)}{p(x)p(y)}\right)
\end{equation*}

where

\begin{equation*}
p(x,y) = \frac{\text{Number of paragraphs with the co-occurrence of x and y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(x) = \frac{\text{Number of paragraphs with the occurrence of x}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(y) = \frac{\text{Number of paragraphs with the occurrence of y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

with the sum over $i$ ranging over all paragraphs. Note that there are other ways PMI could be formulated.

NPMI is defined as:

\begin{equation*}
NPMI = \frac{PMI}{-\log_2(p(x,y))} = \frac{\log_2(p(x)p(y))}{\log_2(p(x,y))} - 1
\end{equation*}

Thus, when there is no co-occurrence, NMPI is -1. NPMI is normalized between [-1, +1].

You should use the same set up as you did to calculate document frequency above: paragraphs as documents, lemmatized, lower-cased, and with term frequency information removed by conversion to Python sets. You need to use the basic method for calculating PMI introduced in class (and also in the reading) which is appropriate for any possible definition of co-occurrence (here, there is co-occurrence when a word pair appears in the same paragraph), but you should only calculate PMI for the words in your test set. You must avoid building the entire co-occurrence matrix, instead you should keeping track of the sums you need for the probabilities as you go along. 

When you have calculated NPMI for all the pairs, you should store the word pair and NPMI-similarity mappings in a dictionary called *NPMI_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *NPMI_similarities*. 

(1 mark)

In [371]:
# NPMI_similarities stores the word pair and NPMI similarity mappings
NPMI_similarities = {}

###
# Your answer BEGINS HERE
###

from math import log2

# function to return number of documents contain both words in a pair
def count_pair(words):
    global wordfreq
    count_pair = 0
    for docid in range(len(wordfreq)):
        wordlist = wordfreq[docid].keys()
        if words[0] in wordlist and words[1] in wordlist:
            count_pair = count_pair + 1
    return count_pair

# total_wordtype stores total number of word types from all documents
total_wordtype = sum([len(wordfreq[docid].keys()) for docid in range(len(wordfreq))])

for keys in final_gold_standard.keys():
    p_x = float(count_word(keys[0]) / total_wordtype)
    p_y = float(count_word(keys[1]) / total_wordtype)
    p_xy = float(count_pair([keys[0],keys[1]]) / total_wordtype)   
    if(p_xy > 0):
        NPMI_similarities[keys] = (log2(p_x*p_y) / log2(p_xy)) - 1
    else:
        NPMI_similarities[keys] = - 1

###
# Your answer ENDS HERE
###

print(NPMI_similarities)

{('bread', 'butter'): 0.6531272737497535, ('professor', 'doctor'): -1, ('student', 'professor'): 0.535495995400926, ('stock', 'egg'): 0.3685128922213745, ('money', 'cash'): 0.44493834723517245, ('king', 'queen'): 0.4181407297139583, ('bishop', 'rabbi'): -1, ('football', 'basketball'): 0.7161994042283006, ('football', 'tennis'): -1, ('alcohol', 'chemistry'): 0.6246376972254175, ('baby', 'mother'): 0.5149353890388499, ('car', 'automobile'): 0.5430334549802616, ('journey', 'voyage'): -1, ('coast', 'shore'): 0.5861510629843374, ('brother', 'monk'): 0.42993185256931743, ('journey', 'car'): -1, ('coast', 'hill'): 0.33931028834846244, ('forest', 'graveyard'): -1, ('monk', 'slave'): -1, ('coast', 'forest'): 0.45787396913892886, ('psychology', 'doctor'): 0.4613281583726412, ('psychology', 'mind'): 0.4461666043435182, ('psychology', 'health'): -1, ('psychology', 'science'): 0.5908740096534864, ('planet', 'moon'): 0.658000616232332, ('planet', 'galaxy'): -1}


<b>For your testing:</b>

In [373]:
assert(NPMI_similarities[('professor', 'doctor')] == -1)

**Instructions:** As PMI matrix is very sparse and can be approximated well by a dense representation via singular value decomposition (SVD), you will derive similarity scores using the Latent Semantic Analysis (LSA) method, i.e. apply SVD and truncate to get a dense vector representation of a word type and then calculate cosine similarity between the two vectors for each word pair. You can use the Distributed Semantics notebook as a starting point, but note that since you are interested here in word semantics, you will be constructing a matrix where the (non-sparse) rows correspond to words in the vocabulary, and the (sparse) columns correspond to the texts where they appear (this is the opposite of the notebook). Again, use the Brown corpus, in the same format as with PMI and document frequency. After you have a matrix in the correct format, use `truncatedSVD` in `sklearn` to produce dense vectors of length k = 500, and then use cosine similarity to produce similarities for your word pairs. 

When you are done, you should store the word pair and LSA-similarity mappings in a dictionary called *LSA_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *LSA_similarities*. 

(1 mark)

In [375]:
# LSA_similarities stores the word pair and LSA similarity mappings
LSA_similarities = {}

###
# Your answer BEGINS HERE
###

from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import TruncatedSVD
from math import sqrt

# build a document term matrix from wordfreq
vectorizer = DictVectorizer()
doc_term_matrix = vectorizer.fit_transform(wordfreq)

# transpose the document term matrix into term document matrix
term_doc_matrix = doc_term_matrix.transpose()

# do the singular value decomposition to get the dense matrix
svd = TruncatedSVD(n_components=500)
matrix_lowrank = svd.fit_transform(term_doc_matrix)

for keys in final_gold_standard.keys():
    index_key1 = vectorizer.feature_names_.index(keys[0])
    index_key2 = vectorizer.feature_names_.index(keys[1])
    vector_key1 = matrix_lowrank[index_key1]
    vector_key2 = matrix_lowrank[index_key2]    
    norm_key1 = sqrt(sum([freq**2 for freq in vector_key1]))
    norm_key2 = sqrt(sum([freq**2 for freq in vector_key2]))   
    dot_product = vector_key1.dot(vector_key2)
    cosine_sim = dot_product / (norm_key1 * norm_key2)
    LSA_similarities[keys] = cosine_sim

###
# Your answer ENDS HERE
###

print(LSA_similarities)

{('bread', 'butter'): 0.14096353694048552, ('professor', 'doctor'): 0.018471068305017425, ('student', 'professor'): 0.11154422247922383, ('stock', 'egg'): 0.02561920802466069, ('money', 'cash'): 0.12209555734199241, ('king', 'queen'): 0.09237189071289414, ('bishop', 'rabbi'): 0.023079327503211962, ('football', 'basketball'): 0.16914930522830596, ('football', 'tennis'): 0.04553614628725452, ('alcohol', 'chemistry'): 0.2510344773287807, ('baby', 'mother'): 0.23044308333640945, ('car', 'automobile'): 0.24807585172503474, ('journey', 'voyage'): 0.11714012186571414, ('coast', 'shore'): 0.3642975409945335, ('brother', 'monk'): 0.01609693696268563, ('journey', 'car'): 0.00419563523740724, ('coast', 'hill'): 0.13518700360739855, ('forest', 'graveyard'): 0.03402865746502399, ('monk', 'slave'): 0.012368708078699362, ('coast', 'forest'): 0.028882880197439786, ('psychology', 'doctor'): 0.08453085528904891, ('psychology', 'mind'): 0.1511329556064947, ('psychology', 'health'): -0.031052414807946863,

<b>For your testing:</b>

In [376]:
assert(LSA_similarities[('professor', 'doctor')] > 0 and LSA_similarities[('professor', 'doctor')] < 0.4)

## 3. Comparison with the Gold Standard (1 mark)


**Instructions:** Finally, you should compare all the similarities you've created to the gold standard you loaded and filtered in the first step. For this, you can use the Pearson correlation co-efficient (`pearsonr`), which is included in scipy (`scipy.stats`). Be careful converting your dictionaries to lists for this purpose, the data for the two datasets needs to be in the same order for correct comparison using correlation. Write a general function, then apply it to each of the similarity score dictionaries.

When you are done, you should put the result in a dictionary called *pearson_correlations* (use the keys: 'lin', 'NPMI', 'LSA'). You may check the section, <i>"For your testing"</i>, below for the expected *pearson_correlations*. 

<b>Hint:</b> All of the methods used here should be markedly above 0, but also far from 1 (perfect correlation); if you're not getting reasonable results, go back and check your code for bugs!  

(1 mark)


In [377]:
from scipy import stats
from scipy.stats import pearsonr

# pearson_correlations stores the pearson correlations with the gold standard of 'lin', 'NPMI', 'LSA'
pearson_correlations = {}

###
# Your answer BEGINS HERE
###

pearson_correlations['lin'] = stats.pearsonr(list(final_gold_standard.values()), list(lin_similarities.values()))[0]
pearson_correlations['NPMI'] = stats.pearsonr(list(final_gold_standard.values()), list(NPMI_similarities.values()))[0]
pearson_correlations['LSA'] = stats.pearsonr(list(final_gold_standard.values()), list(LSA_similarities.values()))[0]

###
# Your answer ENDS HERE
###

print(pearson_correlations)

{'lin': 0.5301489978447533, 'NPMI': 0.1879988886941004, 'LSA': 0.43255544266507806}


<b>For your testing:</b>

In [378]:
assert(pearson_correlations['lin'] > 0.4 and pearson_correlations['lin'] < 0.8)

## Challenge yourself: Improving the correlation (This part will NOT be marked)

You can try to derive a similarity score from word2vec vectors, using the Gensim interface, and compare it with the similarity scores you've created and the gold standard. Check the Gensim word2vec tutorial for details on the API: https://radimrehurek.com/gensim/models/word2vec.html. Again, you should use the Brown for this, but for word2vec you don't need to worry about paragraphs: feel free to train your model at the sentence level instead. Your vectors should have the same number of dimensions as LSA (500), and you need to run for 50 iterations. This may take a while (several minutes).

In [430]:
from gensim.models import Word2Vec

sentences = nltk.corpus.brown.sents()
model = Word2Vec(sentences, min_count=1, size = 500 ,iter = 50)
model.save('brown_model')
new_model = Word2Vec.load('brown_model')

word2vec_similarities = {}

for keys in final_gold_standard.keys():
    word2vec_similarities[keys] = new_model.wv.similarity(keys[0],keys[1])

pearson_correlations['word2vec'] = stats.pearsonr(list(final_gold_standard.values()), list(word2vec_similarities.values()))[0]

print(pearson_correlations['word2vec'])


0.09473714215433966


## A final word

Normally, we would not use a corpus as small as the Brown for the purposes of building distributional word vectors. Also, note that filtering our test set to just words we are likely to do well on would typically be considered cheating.